<a href="https://colab.research.google.com/github/hanocha/try-tensorflow/blob/master/simple_mnist_network_using_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

mnist_test, info_test = tfds.load("mnist", split="test", with_info=True)
mnist_train, info_train = tfds.load("mnist", split="train", with_info=True)

print(mnist_test.take(1))

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/1.0.0. Subsequent calls will reuse this data.
<DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>


In [3]:
import tensorflow as tf
import numpy as np

mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape([60000, -1])
X_test = X_test.reshape([10000, -1])

print(X_train.shape)
print(X_test.shape)

mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train - mean_vals) / std_val
X_test_centered = (X_test - mean_vals) / std_val

print(X_train_centered.shape, y_train.shape)
print(X_test_centered.shape, y_test.shape)

print(y_train)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 784)
(10000, 784)
(60000, 784) (60000,)
(10000, 784) (10000,)
[5 0 4 ... 5 6 8]


In [0]:
import tensorflow as tf

n_features = X_train_centered.shape[1]
n_classes = 10
random_seed = 1234
np.random.seed(random_seed)

g = tf.Graph()

with g.as_default():
  tf.set_random_seed(random_seed)
  tf_x = tf.placeholder(
      dtype=tf.float32,
      shape=(None, n_features),
      name='tf_x'
  )

  tf_y = tf.placeholder(
      dtype=tf.int32,
      shape=None,
      name='tf_y'
  )

  y_onehot = tf.one_hot(indices=tf_y, depth=n_classes)
  
  h1 = tf.layers.dense(
      inputs=tf_x,
      units=50,
      activation=tf.tanh,
      name='layer1',
  )

  h2 = tf.layers.dense(
      inputs=h1,
      units=50,
      activation=tf.tanh,
      name='layer2',
  )

  logits = tf.layers.dense(
      inputs=h2,
      units=10,
      activation=None,
      name='layer3',
  )
  
  predictions = {
      'classes' : tf.argmax(logits, axis=1, name='predicted_classes'),
      'probabilities' : tf.nn.softmax(logits, name='softmax_tensor'),
  }

In [0]:
with g.as_default():
  cost = tf.losses.softmax_cross_entropy(
      onehot_labels=y_onehot,
      logits=logits,
  )

  optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
  train_op = optimizer.minimize(loss=cost)
  init_op = tf.global_variables_initializer()

In [0]:
def create_batch_generator(X, y, batch_size=128, shuffle=False):
  X_copy = np.array(X)
  y_copy = np.array(y)

  if shuffle:
    data = np.column_stack((X_copy, y_copy))
    np.random.shuffle(data)
    X_copy = data[:, :-1]
    y_copy = data[:, -1].astype(int)

  for i in range(0, X.shape[0], batch_size):
    yield (X_copy[i:i+batch_size, :], y_copy[i:i+batch_size])

In [19]:
sess = tf.Session(graph=g)
sess.run(init_op)

training_costs = []

for epoch in range(50):
  training_loss = []
  batch_generator = create_batch_generator(
      X_train_centered,
      y_train,
      batch_size=64,
  )

  for batch_X, batch_y in batch_generator:
    feed = {
        tf_x: batch_X,
        tf_y: batch_y,
    }

    _, batch_cost = sess.run(
        [train_op, cost],
        feed_dict=feed,
    )

    training_costs.append(batch_cost)

  print(epoch + 1)
  print(np.mean(training_costs))

1
1.5789992
2
1.2643682
3
1.0900846
4
0.9745121
5
0.8908125
6
0.82676566
7
0.77583027
8
0.73413855
9
0.69923925
10
0.66949505
11
0.6437669
12
0.6212354
13
0.6012947
14
0.5834865
15
0.567457
16
0.5529286
17
0.5396799
18
0.52753216
19
0.5163393
20
0.50598085
21
0.4963563
22
0.4873811
23
0.47898367
24
0.47110283
25
0.4636861
26
0.4566882
27
0.45006952
28
0.44379574
29
0.43783656
30
0.43216527
31
0.42675832
32
0.42159462
33
0.41665548
34
0.41192406
35
0.40738535
36
0.4030257
37
0.39883283
38
0.39479563
39
0.39090398
40
0.38714868
41
0.38352126
42
0.3800141
43
0.37662005
44
0.37333274
45
0.37014607
46
0.3670546
47
0.3640532
48
0.3611372
49
0.35830218
50
0.35554412


In [20]:
feed = {
    tf_x : X_test_centered,
}

y_pred = sess.run(predictions['classes'], feed_dict=feed)

print(np.sum(y_pred == y_test) / y_test.shape[0]) 

0.9363
